In [1]:
import findspark
findspark.init() # this must be executed before the below import
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark import SparkFiles
import ray
import time
import rtree
from rtree import index
import pandas as pd
import numpy as np
from numpy import genfromtxt
import threading
import pyarrow as pa
import pyarrow.parquet as pq
from partition_tree import PartitionTree

In [2]:
conf = SparkConf().setAll([("spark.executor.memory", "24g"),("spark.driver.memory","24g"),
                           ("spark.memory.offHeap.enabled",True),("spark.memory.offHeap.size","16g"),
                          ("spark.driver.maxResultSize", "16g")])

sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)
import os
os.environ['HADOOP_HOME'] = '/home/liupengju/hadoop'
os.environ['JAVA_HOME'] = '/home/liupengju/java/jdk1.8.0_281'
os.environ['ARROW_LIBHDFS_DIR'] = '/home/liupengju/hadoop/lib/native'

/home/liupengju/anaconda3/envs/torch/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning
/home/liupengju/anaconda3/envs/torch/lib/python3.6/site-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [3]:
def process_chunk_row(row, used_dims, partition_tree, pid_data_dict, count, k):
    if count[0] % 100000 == 0:
        print('proces',k,'has routed',count[0],'rows')
    count[0] += 1
    row_numpy = row.to_numpy()
    row_point = row_numpy[used_dims].tolist()
    pids = [0]
    try:
        pids = partition_tree.get_pid_for_data_point(row_point)
    except:
        print(row_point)
    if isinstance(pids,list):
        for pid in pids:
            if pid in pid_data_dict:
                pid_data_dict[pid]+=[row_numpy.tolist()]
            else:
                pid_data_dict[pid]=[row_numpy.tolist()]

@ray.remote
def process_chunk(chunk, used_dims, partition_path, k, partition_tree):
    print("enter data routing process", k, '..')
    pid_data_dict = {}
    count = [0]
    chunk.apply(lambda row: process_chunk_row(row, used_dims, partition_tree, pid_data_dict, count, k), axis=1)
    dict_id = ray.put(pid_data_dict)
    print("exit data routing process", k, ".")
    return dict_id

@ray.remote
def merge_epochs(parameters):
    #fs = pa.hdfs.connect()
    pids, epoch_count, hdfs_path, fs, merge_process = parameters
    for pid in pids:
        parquets = []
        for epoch in range(epoch_count):
            path = hdfs_path + "epoch_" + str(epoch) + '/partition_' + str(pid)+'.parquet'
            #print(path)
            try:
                par = pq.read_table(path)
                parquets.append(par)
            except:
                continue
        print("process", merge_process, "pid", pid, " len parquets (epochs):", len(parquets))
        if len(parquets) == 0:
            continue
        merged_parquet = pa.concat_tables(parquets)
        merge_path = hdfs_path + 'merged/partition_' + str(pid)+'.parquet'
        with fs.open(merge_path,'wb') as f:
            pq.write_table(merged_parquet, f)
        # fw = fs.open(merge_path,'wb')
        # pq.write_table(merged_parquet, fw)
        # fw.close()
    print('exit merge process', merge_process)

def merge_dict(base_dict, new_dict):
    for key, val in new_dict.items():
        if key in base_dict:
            base_dict[key] += val
        else:
            base_dict[key] = val
    new_dict.clear()

def dump_dict_2_hdfs_epoch(merged_dict, column_names, hdfs_path, fs, epoch):
    #print('= = = start dumping in main thread = = =')
    for pid, val in merged_dict.items():
        #print("writing to pid:",pid)
        path = hdfs_path + 'epoch_'+ str(epoch) +'/partition_' + str(pid) + '.parquet'
        pdf = pd.DataFrame(val, columns=column_names)
        adf = pa.Table.from_pandas(pdf)
        #fw = fs.open(path, 'wb')
        with fs.open(path,'wb') as f:
            pq.write_table(adf, f,write_statistics=False,use_dictionary=False,compression='none')
        # fw = fs.open(path,'wb') # it seems the new version does not have the open function
        # pq.write_table(adf, fw)
        # fw.close()
    #print('= = = exit dumping = = =')


def batch_data_parallel(table_path, partition_path, chunk_size, used_dims, hdfs_path, num_dims, num_process,
                        hdfs_private_ip):
    begin_time = time.time()

    

    # column names for pandas dataframe
    cols = [i for i in range(num_dims)]
    col_names = ['_c' + str(i) for i in range(num_dims)]
    # pyarrow parquent append
    # fs = pa.fs.HadoopFileSystem(hdfs_private_ip, port=9001, user='hdfs', replication=1)
    fs=pa.hdfs.connect(host=hdfs_private_ip, port=9001, user='liupengju')
    partition_tree = PartitionTree(len(used_dims))
    partition_tree.load_tree(partition_path)

    # chunks
    chunk_count = 0
    epoch_count = 0

    # collect object refs
    result_ids = []
    last_batch_ids = []
    first_loop = True

    for chunk in pd.read_table(table_path, delimiter='|', usecols=cols, names=col_names, chunksize=chunk_size):
        print('reading chunk: ', chunk_count)

        chunk_id = ray.put(chunk)
        result_id = process_chunk.remote(chunk_id, used_dims, partition_path, chunk_count, partition_tree)

        del chunk_id
        result_ids.append(result_id)
        del result_id

        # after all process allocated a chunk, process and dump the data
        if chunk_count % num_process == num_process - 1:

            if first_loop:
                first_loop = False
                last_batch_ids = result_ids.copy()
                result_ids.clear()
                chunk_count += 1
                continue
            else:
                print("= = = Process Dump For Chunk", chunk_count - 2 * num_process + 1, "to",
                      chunk_count - num_process, "= = =")
                base_dict = {}
                while len(last_batch_ids):
                    done_id, last_batch_ids = ray.wait(last_batch_ids)
                    dict_id = ray.get(done_id[0])
                    result_dict = ray.get(dict_id)
                    merge_dict(base_dict, result_dict)
                dump_dict_2_hdfs_epoch(base_dict, col_names, hdfs_path, fs, epoch_count)  # consider whether we should use another process
                epoch_count += 1
                base_dict.clear()
                print("= = = Finish Dump For Chunk", chunk_count - 2 * num_process + 1, "to", chunk_count - num_process,"= = =")
                last_batch_ids = result_ids.copy()
                result_ids.clear()

            current_time = time.time()
            time_elapsed = current_time - begin_time
            print("= = = TOTAL PROCESSED SO FAR:", (chunk_count - num_process + 1) * chunk_size, "ROWS. TIME SPENT:",time_elapsed, "SECONDS = = =")

        chunk_count += 1

    # process the last few batches
    print("= = = Process Dump For Last Few Chunks = = =")
    base_dict = {}
    while len(last_batch_ids):
        done_id, last_batch_ids = ray.wait(last_batch_ids)
        dict_id = ray.get(done_id[0])
        result_dict = ray.get(dict_id)
        merge_dict(base_dict, result_dict)
    dump_dict_2_hdfs_epoch(base_dict, col_names, hdfs_path, fs, epoch_count)
    epoch_count += 1
    base_dict.clear()
    last_batch_ids.clear()

    base_dict = {}
    while len(result_ids):
        done_id, result_ids = ray.wait(result_ids)
        dict_id = ray.get(done_id[0])
        result_dict = ray.get(dict_id)
        merge_dict(base_dict, result_dict)
    result_ids.clear()  # clear up the references
    dump_dict_2_hdfs_epoch(base_dict, col_names, hdfs_path, fs, epoch_count)
    epoch_count += 1
    base_dict.clear()
    result_ids.clear()

    # Merge all the epochs
    print("= = = Start Merging the Epochs = = =")
    leaves = partition_tree.get_leaves()
    pids = [leaf.nid for leaf in leaves]
    steps = len(pids) // num_process
    not_ready_ids = []
    for i in range(num_process):
        sub_pids = pids[i * steps:(i + 1) * steps]
        if i == num_process - 1:
            sub_pids = pids[i * steps:]
        rid = merge_epochs.remote([sub_pids, epoch_count, hdfs_path, fs, i])
        not_ready_ids.append(rid)

    while len(not_ready_ids):
        ready_ids, not_ready_ids = ray.wait(not_ready_ids)

    
    # Todo: delete temporary epoch parquet data
    print("= = = Delete the used Epochs = = =")
    for epoch in range(epoch_count):
        del_path=hdfs_path + "epoch_" + str(epoch)
        if fs.exists(del_path):
            fs.rm(del_path,recursive=True)
        print("delete epoch_"+str(epoch))
    finish_time = time.time()
    print('= = = = = TOTAL DATA ROUTING AND PERISITING TIME:', finish_time - begin_time, "= = = = =")


In [4]:
# = = = Configuration (UBDA Cloud Centos) = = =
scale_factor = 50
# table_base_path = '/media/datadrive1/TPCH/dbgen/'
# table_path = table_base_path + 'lineitem_' + str(scale_factor) + '.tbl'
# table_path='/home/liupengju/pycharmProjects/NORA_JOIN_SIMULATION/NORA_experiments/dataset/lineitem_1.tbl'
table_path='/home/liupengju/pycharmProjects/NORA_JOIN_SIMULATION/NORA_experiments/dataset/store_sales_50.tbl'
num_process = 10
chunk_size = 2000000
# 6M rows = about 1GB raw data

num_dims = 16
# used_dims = [1, 2, 4] #lineitem
used_dims = [1, 2, 3] #store_sales

# hdfs_path: whole table data is partitioned into these parquet files.
# base path of HDFS
hdfs_private_ip = '10.77.110.133'
# hdfs_base_path = 'hdfs://10.77.110.133:9001/par_nora/'
hdfs_base_path = 'hdfs://10.77.110.133:9001/par_nora/tpcds/'
problem_type = 2
# nora_hdfs = hdfs_base_path + 'NORA/prob' + str(problem_type) + '/'
# qdtree_hdfs = hdfs_base_path + 'QdTree/prob' + str(problem_type) + '/'
# kdtree_hdfs = hdfs_base_path + 'KDTree/prob' + str(problem_type) + '/'

nora_hdfs = hdfs_base_path + 'NORA/prob' + str(problem_type) + '/scale' + str(scale_factor) + "/"
qdtree_hdfs = hdfs_base_path + 'QdTree/prob' + str(problem_type) + '/scale' + str(scale_factor) + "/"
paw_hdfs = hdfs_base_path + 'PAW/prob' + str(problem_type) + '/scale' + str(scale_factor) + "/"

# === The partition tree files ====
# base path of Partition
# partition_base_path = '/home/liupengju/pycharmProjects/NORA_JOIN_SIMULATION/PartitionLayout/'
partition_base_path = '/home/liupengju/pycharmProjects/NORA_JOIN_SIMULATION/PartitionLayout/tpcds/'
# partition_path: the location of constructed partition tree
# nora_partition = partition_base_path + 'prob' + str(problem_type) + '_nora'
# qdtree_partition = partition_base_path + 'prob' + str(problem_type) + '_qdtree'
# kdtree_partition = partition_base_path + 'prob' + str(problem_type) + '_kdtree'
nora_partition = partition_base_path + 'prob' + str(problem_type) + '_nora_scale' + str(scale_factor)
qdtree_partition = partition_base_path + 'prob' + str(problem_type) + '_qdtree_scale' + str(scale_factor)
paw_partition = partition_base_path + 'prob' + str(problem_type) + '_paw_scale' + str(scale_factor)

In [5]:
# batch_data_parallel(table_path, nora_partition, chunk_size, used_dims, nora_hdfs, num_dims, num_process, hdfs_private_ip)
# print('finish nora data routing..')
# batch_data_parallel(table_path, paw_partition, chunk_size, used_dims, paw_hdfs, num_dims, num_process, hdfs_private_ip)
# print('finish paw data routing..')
# time.sleep(10000)
# batch_data_parallel(table_path, qdtree_partition, chunk_size, used_dims, qdtree_hdfs, num_dims, num_process, hdfs_private_ip)
# print('finish qdtree data routing..')

In [6]:
ray.init(num_cpus=num_process)

2022-08-29 16:58:23,787	INFO services.py:1173 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '10.77.110.133',
 'raylet_ip_address': '10.77.110.133',
 'redis_address': '10.77.110.133:6379',
 'object_store_address': '/tmp/ray/session_2022-08-29_16-58-23_184515_731913/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-08-29_16-58-23_184515_731913/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2022-08-29_16-58-23_184515_731913',
 'metrics_export_port': 55519,
 'node_id': 'f3d79cf4d60edc5ce32980fe98225b8f26b9ee97'}

In [7]:
batch_data_parallel(table_path, qdtree_partition, chunk_size, used_dims, qdtree_hdfs, num_dims, num_process, hdfs_private_ip)
print('finish qdtree data routing..')

/home/liupengju/anaconda3/envs/torch/lib/python3.6/site-packages/ipykernel_launcher.py:90: FutureWarning: pyarrow.hdfs.connect is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.


reading chunk:  0
(pid=732385) enter data routing process 0 ..
(pid=732385) proces 0 has routed 0 rows
reading chunk:  1
(pid=732384) enter data routing process 1 ..
(pid=732384) proces 1 has routed 0 rows
reading chunk:  2
(pid=732388) enter data routing process 2 ..
(pid=732388) proces 2 has routed 0 rows
(pid=732385) proces 0 has routed 100000 rows
reading chunk:  3
(pid=732384) proces 1 has routed 100000 rows
(pid=732387) enter data routing process 3 ..
(pid=732387) proces 3 has routed 0 rows
reading chunk:  4
(pid=732385) proces 0 has routed 200000 rows
(pid=732388) proces 2 has routed 100000 rows
(pid=732380) enter data routing process 4 ..
(pid=732380) proces 4 has routed 0 rows
reading chunk:  5
(pid=732384) proces 1 has routed 200000 rows
(pid=732387) proces 3 has routed 100000 rows
(pid=732379) enter data routing process 5 ..
(pid=732379) proces 5 has routed 0 rows
reading chunk:  6
(pid=732385) proces 0 has routed 300000 rows
(pid=732388) proces 2 has routed 200000 rows
(pid

(pid=732379) proces 5 has routed 1200000 rows
(pid=732382) proces 9 has routed 1000000 rows
(pid=732381) proces 7 has routed 1100000 rows
(pid=732380) proces 4 has routed 1300000 rows
(pid=732388) proces 2 has routed 1400000 rows
(pid=732383) proces 6 has routed 1200000 rows
(pid=732385) proces 0 has routed 1500000 rows
(pid=732386) proces 8 has routed 1100000 rows
(pid=732387) proces 3 has routed 1400000 rows
(pid=732384) proces 1 has routed 1500000 rows
(pid=732379) proces 5 has routed 1300000 rows
(pid=732382) proces 9 has routed 1100000 rows
(pid=732381) proces 7 has routed 1200000 rows
(pid=732380) proces 4 has routed 1400000 rows
(pid=732383) proces 6 has routed 1300000 rows
(pid=732388) proces 2 has routed 1500000 rows
(pid=732385) proces 0 has routed 1600000 rows
(pid=732386) proces 8 has routed 1200000 rows
(pid=732384) proces 1 has routed 1600000 rows
(pid=732387) proces 3 has routed 1500000 rows
(pid=732379) proces 5 has routed 1400000 rows
(pid=732382) proces 9 has routed 1

(pid=732382) proces 19 has routed 200000 rows
(pid=732380) proces 14 has routed 500000 rows
(pid=732385) proces 10 has routed 700000 rows
(pid=732383) proces 16 has routed 400000 rows
(pid=732381) proces 17 has routed 300000 rows
(pid=732387) proces 12 has routed 600000 rows
(pid=732388) proces 13 has routed 600000 rows
(pid=732384) proces 11 has routed 700000 rows
(pid=732379) proces 15 has routed 500000 rows
(pid=732386) proces 18 has routed 300000 rows
(pid=732382) proces 19 has routed 300000 rows
(pid=732385) proces 10 has routed 800000 rows
(pid=732380) proces 14 has routed 600000 rows
(pid=732383) proces 16 has routed 500000 rows
(pid=732381) proces 17 has routed 400000 rows
(pid=732387) proces 12 has routed 700000 rows
(pid=732384) proces 11 has routed 800000 rows
(pid=732388) proces 13 has routed 700000 rows
(pid=732379) proces 15 has routed 600000 rows
(pid=732386) proces 18 has routed 400000 rows
(pid=732382) proces 19 has routed 400000 rows
(pid=732380) proces 14 has routed 

(pid=732384) exit data routing process 11 .
(pid=732379) proces 15 has routed 1900000 rows
(pid=732382) proces 19 has routed 1700000 rows
(pid=732386) proces 18 has routed 1700000 rows
(pid=732383) proces 16 has routed 1900000 rows
reading chunk:  20
(pid=732384) enter data routing process 20 ..
(pid=732384) proces 20 has routed 0 rows
(pid=732381) proces 17 has routed 1800000 rows
reading chunk:  21
(pid=732385) enter data routing process 21 ..
(pid=732385) proces 21 has routed 0 rows
(pid=732387) exit data routing process 12 .
(pid=732388) exit data routing process 13 .
(pid=732386) proces 18 has routed 1800000 rows
(pid=732382) proces 19 has routed 1800000 rows
reading chunk:  22
(pid=732384) proces 20 has routed 100000 rows
(pid=732387) enter data routing process 22 ..
(pid=732380) exit data routing process 14 .
(pid=732387) proces 22 has routed 0 rows
(pid=732385) proces 21 has routed 100000 rows
(pid=732381) proces 17 has routed 1900000 rows
reading chunk:  23
(pid=732388) enter 

...

(pid=732387) proces 117 has routed 200000 rows
(pid=732382) proces 111 has routed 500000 rows
(pid=732380) proces 115 has routed 300000 rows
(pid=732386) proces 113 has routed 400000 rows
(pid=732381) proces 112 has routed 500000 rows
(pid=732379) proces 116 has routed 300000 rows
(pid=732383) proces 114 has routed 400000 rows
(pid=732384) proces 110 has routed 600000 rows
(pid=732387) proces 117 has routed 300000 rows
(pid=732380) proces 115 has routed 400000 rows
(pid=732382) proces 111 has routed 600000 rows
(pid=732386) proces 113 has routed 500000 rows
(pid=732383) proces 114 has routed 500000 rows
(pid=732384) proces 110 has routed 700000 rows
(pid=732381) proces 112 has routed 600000 rows
(pid=732379) proces 116 has routed 400000 rows
(pid=732387) proces 117 has routed 400000 rows
(pid=732380) proces 115 has routed 500000 rows
(pid=732382) proces 111 has routed 700000 rows
(pid=732386) proces 113 has routed 600000 rows
(pid=732383) proces 114 has routed 600000 rows
(pid=732384) 

(pid=732384) /home/liupengju/anaconda3/envs/torch/lib/python3.6/site-packages/ray/serialization.py:216: FutureWarning: pyarrow.hdfs.HadoopFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
(pid=732384)   obj = pickle.loads(in_band)
(pid=732382) /home/liupengju/anaconda3/envs/torch/lib/python3.6/site-packages/ray/serialization.py:216: FutureWarning: pyarrow.hdfs.HadoopFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
(pid=732382)   obj = pickle.loads(in_band)
(pid=732386) /home/liupengju/anaconda3/envs/torch/lib/python3.6/site-packages/ray/serialization.py:216: FutureWarning: pyarrow.hdfs.HadoopFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
(pid=732386)   obj = pickle.loads(in_band)
(pid=732379) /home/liupengju/anaconda3/envs/torch/lib/python3.6/site-packages/ray/serialization.py:216: FutureWarning: pyarrow.hdfs.HadoopFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.H

(pid=732388) process 8 pid 476  len parquets (epochs): 12
(pid=732386) process 3 pid 165  len parquets (epochs): 12
(pid=732384) process 6 pid 347  len parquets (epochs): 12
(pid=732380) process 1 pid 137  len parquets (epochs): 12
(pid=732383) process 4 pid 176  len parquets (epochs): 12
(pid=732385) process 9 pid 397  len parquets (epochs): 12
(pid=732381) process 5 pid 452  len parquets (epochs): 12
(pid=732387) process 2 pid 151  len parquets (epochs): 12
(pid=732382) process 7 pid 216  len parquets (epochs): 12
(pid=732379) process 0 pid 253  len parquets (epochs): 12
(pid=732386) process 3 pid 166  len parquets (epochs): 12
(pid=732384) process 6 pid 348  len parquets (epochs): 12
(pid=732388) process 8 pid 477  len parquets (epochs): 12
(pid=732380) process 1 pid 271  len parquets (epochs): 12
(pid=732383) process 4 pid 177  len parquets (epochs): 12
(pid=732381) process 5 pid 453  len parquets (epochs): 12
(pid=732385) process 9 pid 481  len parquets (epochs): 12
(pid=732387) p

(pid=732386) process 3 pid 442  len parquets (epochs): 12
(pid=732380) process 1 pid 280  len parquets (epochs): 12
(pid=732379) process 0 pid 261  len parquets (epochs): 12
(pid=732387) process 2 pid 296  len parquets (epochs): 12
(pid=732381) process 5 pid 339  len parquets (epochs): 12
(pid=732382) process 7 pid 370  len parquets (epochs): 12
(pid=732383) process 4 pid 326  len parquets (epochs): 12
(pid=732386) process 3 pid 443  len parquets (epochs): 12
(pid=732384) process 6 pid 207  len parquets (epochs): 12
(pid=732379) process 0 pid 262  len parquets (epochs): 12
(pid=732380) process 1 pid 143  len parquets (epochs): 12
(pid=732381) process 5 pid 340  len parquets (epochs): 12
(pid=732387) process 2 pid 297  len parquets (epochs): 12
(pid=732385) process 9 pid 407  len parquets (epochs): 12
(pid=732388) process 8 pid 233  len parquets (epochs): 12
(pid=732382) process 7 pid 222  len parquets (epochs): 12
(pid=732383) process 4 pid 449  len parquets (epochs): 12
(pid=732386) p

(pid=732379) process 0 pid 269  len parquets (epochs): 12
(pid=732387) process 2 pid 163  len parquets (epochs): 12
(pid=732381) exit merge process 5
(pid=732382) process 7 pid 492  len parquets (epochs): 12
(pid=732380) process 1 pid 285  len parquets (epochs): 12
(pid=732383) process 4 pid 187  len parquets (epochs): 12
(pid=732382) exit merge process 7
(pid=732384) process 6 pid 362  len parquets (epochs): 12
(pid=732385) process 9 pid 418  len parquets (epochs): 12
(pid=732384) exit merge process 6
(pid=732379) process 0 pid 429  len parquets (epochs): 12
(pid=732383) process 4 pid 188  len parquets (epochs): 12
(pid=732387) process 2 pid 164  len parquets (epochs): 12
(pid=732380) process 1 pid 286  len parquets (epochs): 12
(pid=732385) process 9 pid 419  len parquets (epochs): 12
(pid=732380) process 1 pid 150  len parquets (epochs): 12
(pid=732387) exit merge process 2
(pid=732383) process 4 pid 189  len parquets (epochs): 12
(pid=732379) process 0 pid 430  len parquets (epochs

In [8]:
batch_data_parallel(table_path, paw_partition, chunk_size, used_dims, paw_hdfs, num_dims, num_process, hdfs_private_ip)
print('finish paw data routing..')

/home/liupengju/anaconda3/envs/torch/lib/python3.6/site-packages/ipykernel_launcher.py:90: FutureWarning: pyarrow.hdfs.connect is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.


reading chunk:  0
(pid=732383) enter data routing process 0 ..
(pid=732383) proces 0 has routed 0 rows
reading chunk:  1
(pid=732385) enter data routing process 1 ..
(pid=732385) proces 1 has routed 0 rows
reading chunk:  2
(pid=732379) enter data routing process 2 ..
(pid=732379) proces 2 has routed 0 rows
(pid=732383) proces 0 has routed 100000 rows
reading chunk:  3
(pid=732380) enter data routing process 3 ..
(pid=732380) proces 3 has routed 0 rows
(pid=732385) proces 1 has routed 100000 rows
(pid=732383) proces 0 has routed 200000 rows
(pid=732379) proces 2 has routed 100000 rows
reading chunk:  4
(pid=732387) enter data routing process 4 ..
(pid=732387) proces 4 has routed 0 rows
(pid=732380) proces 3 has routed 100000 rows
(pid=732385) proces 1 has routed 200000 rows
reading chunk:  5
(pid=732384) enter data routing process 5 ..
(pid=732384) proces 5 has routed 0 rows
(pid=732379) proces 2 has routed 200000 rows
(pid=732383) proces 0 has routed 300000 rows
reading chunk:  6
(pid

(pid=732384) proces 5 has routed 1200000 rows
(pid=732381) proces 7 has routed 1100000 rows
(pid=732379) proces 2 has routed 1400000 rows
(pid=732386) proces 8 has routed 1100000 rows
(pid=732388) proces 9 has routed 1100000 rows
(pid=732383) proces 0 has routed 1400000 rows
(pid=732385) proces 1 has routed 1500000 rows
(pid=732382) proces 6 has routed 1200000 rows
(pid=732387) proces 4 has routed 1200000 rows
(pid=732384) proces 5 has routed 1300000 rows
(pid=732380) proces 3 has routed 1400000 rows
(pid=732381) proces 7 has routed 1200000 rows
(pid=732379) proces 2 has routed 1500000 rows
(pid=732386) proces 8 has routed 1200000 rows
(pid=732383) proces 0 has routed 1500000 rows
(pid=732388) proces 9 has routed 1200000 rows
(pid=732385) proces 1 has routed 1600000 rows
(pid=732387) proces 4 has routed 1300000 rows
(pid=732382) proces 6 has routed 1300000 rows
(pid=732380) proces 3 has routed 1500000 rows
(pid=732384) proces 5 has routed 1400000 rows
(pid=732381) proces 7 has routed 1

(pid=732380) proces 13 has routed 500000 rows
(pid=732385) proces 10 has routed 800000 rows
(pid=732388) proces 15 has routed 400000 rows
(pid=732382) proces 16 has routed 400000 rows
(pid=732383) proces 12 has routed 600000 rows
(pid=732381) proces 18 has routed 300000 rows
(pid=732387) proces 19 has routed 300000 rows
(pid=732386) proces 17 has routed 300000 rows
(pid=732379) proces 11 has routed 700000 rows
(pid=732388) proces 15 has routed 500000 rows
(pid=732385) proces 10 has routed 900000 rows
(pid=732382) proces 16 has routed 500000 rows
(pid=732384) proces 14 has routed 500000 rows
(pid=732380) proces 13 has routed 600000 rows
(pid=732381) proces 18 has routed 400000 rows
(pid=732383) proces 12 has routed 700000 rows
(pid=732387) proces 19 has routed 400000 rows
(pid=732386) proces 17 has routed 400000 rows
(pid=732385) proces 10 has routed 1000000 rows
(pid=732388) proces 15 has routed 600000 rows
(pid=732379) proces 11 has routed 800000 rows
(pid=732382) proces 16 has routed

(pid=732380) proces 13 has routed 1900000 rows
(pid=732384) proces 14 has routed 1800000 rows
(pid=732386) proces 17 has routed 1700000 rows
(pid=732382) proces 16 has routed 1800000 rows
(pid=732388) proces 15 has routed 1900000 rows
= = = Finish Dump For Chunk(pid=732381) proces 18 has routed 1700000 rows
(pid=732387) proces 19 has routed 1700000 rows
(pid=732384) proces 14 has routed 1900000 rows
(pid=732383) exit data routing process 12 .
(pid=732382) proces 16 has routed 1900000 rows
(pid=732379) exit data routing process 11 .
 (pid=732386) proces 17 has routed 1800000 rows
(pid=732381) proces 18 has routed 1800000 rows
(pid=732387) proces 19 has routed 1800000 rows
(pid=732380) exit data routing process 13 .
0 to 9 = = =
= = = TOTAL PROCESSED SO FAR: 20000000 ROWS. TIME SPENT: 294.6706614494324 SECONDS = = =
(pid=732386) proces 17 has routed 1900000 rows
(pid=732381) proces 18 has routed 1900000 rows
reading chunk:  20
(pid=732380) enter data routing process 20 ..
(pid=732380) pr

...

(pid=732379) proces 108 has routed 900000 rows
(pid=732380) proces 100 has routed 1300000 rows
(pid=732387) proces 107 has routed 1000000 rows
(pid=732382) proces 104 has routed 1100000 rows
(pid=732384) proces 103 has routed 1200000 rows
(pid=732386) proces 101 has routed 1300000 rows
(pid=732381) proces 109 has routed 900000 rows
(pid=732388) proces 102 has routed 1200000 rows
(pid=732383) proces 106 has routed 1000000 rows
(pid=732385) proces 105 has routed 1100000 rows
(pid=732380) proces 100 has routed 1400000 rows
(pid=732379) proces 108 has routed 1000000 rows
(pid=732387) proces 107 has routed 1100000 rows
(pid=732384) proces 103 has routed 1300000 rows
(pid=732382) proces 104 has routed 1200000 rows
(pid=732386) proces 101 has routed 1400000 rows
(pid=732381) proces 109 has routed 1000000 rows
(pid=732388) proces 102 has routed 1300000 rows
(pid=732383) proces 106 has routed 1100000 rows
(pid=732385) proces 105 has routed 1200000 rows
(pid=732379) proces 108 has routed 1100000

(pid=732384) proces 117 has routed 100000 rows
(pid=732387) proces 113 has routed 300000 rows
(pid=732381) proces 111 has routed 400000 rows
(pid=732382) proces 115 has routed 200000 rows
(pid=732379) proces 112 has routed 400000 rows
(pid=732388) proces 116 has routed 200000 rows
(pid=732385) proces 114 has routed 300000 rows
(pid=732383) proces 110 has routed 500000 rows
(pid=732384) proces 117 has routed 200000 rows
(pid=732387) proces 113 has routed 400000 rows
(pid=732381) proces 111 has routed 500000 rows
(pid=732382) proces 115 has routed 300000 rows
(pid=732379) proces 112 has routed 500000 rows
(pid=732388) proces 116 has routed 300000 rows
(pid=732385) proces 114 has routed 400000 rows
(pid=732383) proces 110 has routed 600000 rows
(pid=732384) proces 117 has routed 300000 rows
(pid=732387) proces 113 has routed 500000 rows
(pid=732381) proces 111 has routed 600000 rows
(pid=732382) proces 115 has routed 400000 rows
(pid=732379) proces 112 has routed 600000 rows
(pid=732388) 

(pid=732388) proces 116 has routed 1800000 rows
(pid=732387) exit data routing process 113 .
(pid=732388) proces 116 has routed 1900000 rows
(pid=732384) exit data routing process 117 .
(pid=732385) exit data routing process 114 .
(pid=732382) exit data routing process 115 .
(pid=732388) exit data routing process 116 .
= = = Start Merging the Epochs = = =
(pid=732385) process 2 pid 289  len parquets (epochs): 12
(pid=732387) process 4 pid 325  len parquets (epochs): 12
(pid=732380) process 9 pid 516  len parquets (epochs): 12
(pid=732386) process 8 pid 529  len parquets (epochs): 12
(pid=732382) process 1 pid 448  len parquets (epochs): 12
(pid=732381) process 7 pid 214  len parquets (epochs): 12
(pid=732384) process 3 pid 163  len parquets (epochs): 12
(pid=732383) process 6 pid 200  len parquets (epochs): 12
(pid=732385) process 2 pid 290  len parquets (epochs): 12
(pid=732386) process 8 pid 530  len parquets (epochs): 12
(pid=732388) process 0 pid 253  len parquets (epochs): 12
(pid

(pid=732380) process 9 pid 417  len parquets (epochs): 12
(pid=732386) process 8 pid 398  len parquets (epochs): 12
(pid=732388) process 0 pid 262  len parquets (epochs): 12
(pid=732381) process 7 pid 500  len parquets (epochs): 12
(pid=732384) process 3 pid 466  len parquets (epochs): 12
(pid=732387) process 4 pid 334  len parquets (epochs): 12
(pid=732379) process 5 pid 484  len parquets (epochs): 12
(pid=732383) process 6 pid 363  len parquets (epochs): 12
(pid=732382) process 1 pid 280  len parquets (epochs): 12
(pid=732385) process 2 pid 298  len parquets (epochs): 12
(pid=732380) process 9 pid 418  len parquets (epochs): 12
(pid=732386) process 8 pid 399  len parquets (epochs): 12
(pid=732388) process 0 pid 263  len parquets (epochs): 12
(pid=732381) process 7 pid 379  len parquets (epochs): 12
(pid=732383) process 6 pid 364  len parquets (epochs): 12
(pid=732384) process 3 pid 315  len parquets (epochs): 12
(pid=732387) process 4 pid 182  len parquets (epochs): 12
(pid=732379) p

(pid=732385) process 2 pid 307  len parquets (epochs): 12
(pid=732384) process 3 pid 322  len parquets (epochs): 12
(pid=732388) process 0 pid 270  len parquets (epochs): 12
(pid=732383) process 6 pid 211  len parquets (epochs): 12
(pid=732381) process 7 pid 506  len parquets (epochs): 12
(pid=732380) process 9 pid 521  len parquets (epochs): 12
(pid=732386) process 8 pid 514  len parquets (epochs): 12
(pid=732385) process 2 pid 308  len parquets (epochs): 12
(pid=732379) process 5 pid 351  len parquets (epochs): 12
(pid=732387) exit merge process 4
(pid=732388) process 0 pid 445  len parquets (epochs): 12
(pid=732384) process 3 pid 323  len parquets (epochs): 12
(pid=732383) process 6 pid 212  len parquets (epochs): 12
(pid=732382) process 1 pid 285  len parquets (epochs): 12
(pid=732380) process 9 pid 522  len parquets (epochs): 12
(pid=732381) process 7 pid 387  len parquets (epochs): 12
(pid=732379) process 5 pid 489  len parquets (epochs): 12
(pid=732385) process 2 pid 162  len pa

In [9]:
batch_data_parallel(table_path, nora_partition, chunk_size, used_dims, nora_hdfs, num_dims, num_process, hdfs_private_ip)
print('finish nora data routing..')

/home/liupengju/anaconda3/envs/torch/lib/python3.6/site-packages/ipykernel_launcher.py:90: FutureWarning: pyarrow.hdfs.connect is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.


reading chunk:  0
(pid=732380) enter data routing process 0 ..
(pid=732380) proces 0 has routed 0 rows
reading chunk:  1
(pid=732379) enter data routing process 1 ..
(pid=732379) proces 1 has routed 0 rows
reading chunk:  2
(pid=732382) enter data routing process 2 ..
(pid=732380) proces 0 has routed 100000 rows
(pid=732382) proces 2 has routed 0 rows
reading chunk:  3
(pid=732388) enter data routing process 3 ..
(pid=732388) proces 3 has routed 0 rows
(pid=732379) proces 1 has routed 100000 rows
reading chunk:  4
(pid=732383) enter data routing process 4 ..
(pid=732383) proces 4 has routed 0 rows
(pid=732380) proces 0 has routed 200000 rows
(pid=732382) proces 2 has routed 100000 rows
reading chunk:  5
(pid=732381) enter data routing process 5 ..
(pid=732381) proces 5 has routed 0 rows
(pid=732379) proces 1 has routed 200000 rows
(pid=732388) proces 3 has routed 100000 rows
reading chunk:  6
(pid=732384) enter data routing process 6 ..
(pid=732384) proces 6 has routed 0 rows
(pid=7323

(pid=732380) proces 0 has routed 1400000 rows
(pid=732387) proces 9 has routed 1000000 rows
(pid=732382) proces 2 has routed 1300000 rows
(pid=732385) proces 7 has routed 1100000 rows
(pid=732386) proces 8 has routed 1100000 rows
(pid=732379) proces 1 has routed 1400000 rows
(pid=732388) proces 3 has routed 1300000 rows
(pid=732381) proces 5 has routed 1200000 rows
(pid=732384) proces 6 has routed 1200000 rows
(pid=732383) proces 4 has routed 1300000 rows
(pid=732387) proces 9 has routed 1100000 rows
(pid=732380) proces 0 has routed 1500000 rows
(pid=732382) proces 2 has routed 1400000 rows
(pid=732385) proces 7 has routed 1200000 rows
(pid=732386) proces 8 has routed 1200000 rows
(pid=732379) proces 1 has routed 1500000 rows
(pid=732388) proces 3 has routed 1400000 rows
(pid=732381) proces 5 has routed 1300000 rows
(pid=732384) proces 6 has routed 1300000 rows
(pid=732387) proces 9 has routed 1200000 rows
(pid=732380) proces 0 has routed 1600000 rows
(pid=732383) proces 4 has routed 1

(pid=732385) proces 17 has routed 300000 rows
(pid=732386) proces 19 has routed 200000 rows
(pid=732379) proces 11 has routed 600000 rows
(pid=732382) proces 12 has routed 500000 rows
(pid=732381) proces 15 has routed 400000 rows
(pid=732384) proces 16 has routed 400000 rows
(pid=732388) proces 13 has routed 500000 rows
(pid=732387) proces 18 has routed 300000 rows
(pid=732383) proces 14 has routed 500000 rows
(pid=732380) proces 10 has routed 700000 rows
(pid=732385) proces 17 has routed 400000 rows
(pid=732386) proces 19 has routed 300000 rows
(pid=732379) proces 11 has routed 700000 rows
(pid=732382) proces 12 has routed 600000 rows
(pid=732381) proces 15 has routed 500000 rows
(pid=732384) proces 16 has routed 500000 rows
(pid=732388) proces 13 has routed 600000 rows
(pid=732387) proces 18 has routed 400000 rows
(pid=732383) proces 14 has routed 600000 rows
(pid=732380) proces 10 has routed 800000 rows
(pid=732385) proces 17 has routed 500000 rows
(pid=732386) proces 19 has routed 

(pid=732383) proces 14 has routed 1800000 rows
(pid=732384) proces 16 has routed 1800000 rows
(pid=732385) proces 17 has routed 1700000 rows
(pid=732382) proces 12 has routed 1900000 rows
(pid=732381) proces 15 has routed 1800000 rows
(pid=732386) proces 19 has routed 1600000 rows
(pid=732388) proces 13 has routed 1900000 rows
(pid=732387) proces 18 has routed 1700000 rows
(pid=732380) exit data routing process 10 .
(pid=732383) proces 14 has routed 1900000 rows
(pid=732379) exit data routing process 11 .
(pid=732384) proces 16 has routed 1900000 rows
(pid=732385) proces 17 has routed 1800000 rows
(pid=732381) proces 15 has routed 1900000 rows
(pid=732386) proces 19 has routed 1700000 rows
(pid=732387) proces 18 has routed 1800000 rows
(pid=732385) proces 17 has routed 1900000 rows
(pid=732386) proces 19 has routed 1800000 rows
(pid=732382) exit data routing process 12 .
(pid=732387) proces 18 has routed 1900000 rows
(pid=732388) exit data routing process 13 .
(pid=732383) exit data ro

...

(pid=732384) proces 116 has routed 100000 rows
(pid=732380) proces 110 has routed 400000 rows
(pid=732381) proces 114 has routed 200000 rows
(pid=732379) proces 112 has routed 300000 rows
(pid=732385) proces 117 has routed 100000 rows
(pid=732383) proces 115 has routed 200000 rows
(pid=732388) proces 111 has routed 400000 rows
(pid=732382) proces 113 has routed 300000 rows
(pid=732384) proces 116 has routed 200000 rows
(pid=732381) proces 114 has routed 300000 rows
(pid=732380) proces 110 has routed 500000 rows
(pid=732379) proces 112 has routed 400000 rows
(pid=732385) proces 117 has routed 200000 rows
(pid=732383) proces 115 has routed 300000 rows
(pid=732388) proces 111 has routed 500000 rows
(pid=732382) proces 113 has routed 400000 rows
(pid=732384) proces 116 has routed 300000 rows
(pid=732381) proces 114 has routed 400000 rows
(pid=732380) proces 110 has routed 600000 rows
(pid=732379) proces 112 has routed 500000 rows
(pid=732385) proces 117 has routed 300000 rows
(pid=732383) 

(pid=732388) exit data routing process 111 .
(pid=732385) proces 117 has routed 1800000 rows
(pid=732383) proces 115 has routed 1900000 rows
(pid=732380) exit data routing process 110 .
(pid=732384) proces 116 has routed 1900000 rows
(pid=732382) exit data routing process 113 .
(pid=732379) exit data routing process 112 .
(pid=732385) exit data routing process 117 .
(pid=732381) exit data routing process 114 .
(pid=732383) exit data routing process 115 .
(pid=732384) exit data routing process 116 .
= = = Start Merging the Epochs = = =
(pid=732380) process 6 pid 353  len parquets (epochs): 12
(pid=732379) process 4 pid 489  len parquets (epochs): 12
(pid=732386) process 8 pid 390  len parquets (epochs): 12
(pid=732385) process 3 pid 479  len parquets (epochs): 12
(pid=732384) process 0 pid 627  len parquets (epochs): 12
(pid=732383) process 1 pid 457  len parquets (epochs): 12
(pid=732382) process 5 pid 569  len parquets (epochs): 12
(pid=732388) process 7 pid 372  len parquets (epochs)

(pid=732385) process 3 pid 303  len parquets (epochs): 12
(pid=732382) process 5 pid 500  len parquets (epochs): 12
(pid=732379) process 4 pid 178  len parquets (epochs): 12
(pid=732384) process 0 pid 446  len parquets (epochs): 12
(pid=732381) process 2 pid 150  len parquets (epochs): 12
(pid=732380) process 6 pid 511  len parquets (epochs): 12
(pid=732386) process 8 pid 711  len parquets (epochs): 12
(pid=732388) process 7 pid 685  len parquets (epochs): 12
(pid=732387) process 9 pid 421  len parquets (epochs): 12
(pid=732383) process 1 pid 647  len parquets (epochs): 12
(pid=732382) process 5 pid 679  len parquets (epochs): 12
(pid=732379) process 4 pid 325  len parquets (epochs): 12
(pid=732385) process 3 pid 663  len parquets (epochs): 12
(pid=732384) process 0 pid 447  len parquets (epochs): 12
(pid=732381) process 2 pid 591  len parquets (epochs): 12
(pid=732380) process 6 pid 512  len parquets (epochs): 12
(pid=732388) process 7 pid 686  len parquets (epochs): 12
(pid=732382) p

(pid=732383) process 1 pid 279  len parquets (epochs): 12
(pid=732387) process 9 pid 621  len parquets (epochs): 12
(pid=732384) process 0 pid 451  len parquets (epochs): 12
(pid=732388) process 7 pid 690  len parquets (epochs): 12
(pid=732381) process 2 pid 292  len parquets (epochs): 12
(pid=732386) process 8 pid 405  len parquets (epochs): 12
(pid=732380) process 6 pid 519  len parquets (epochs): 12
(pid=732382) process 5 pid 346  len parquets (epochs): 12
(pid=732385) process 3 pid 672  len parquets (epochs): 12
(pid=732379) process 4 pid 598  len parquets (epochs): 12
(pid=732383) process 1 pid 280  len parquets (epochs): 12
(pid=732388) process 7 pid 691  len parquets (epochs): 12
(pid=732387) process 9 pid 622  len parquets (epochs): 12
(pid=732384) process 0 pid 452  len parquets (epochs): 12
(pid=732380) process 6 pid 520  len parquets (epochs): 12
(pid=732381) process 2 pid 659  len parquets (epochs): 12
(pid=732382) process 5 pid 347  len parquets (epochs): 12
(pid=732386) p

(pid=732382) exit merge process 5
(pid=732384) process 0 pid 550  len parquets (epochs): 12
(pid=732380) process 6 pid 610  len parquets (epochs): 12
(pid=732381) process 2 pid 478  len parquets (epochs): 12
(pid=732385) process 3 pid 316  len parquets (epochs): 12
(pid=732383) exit merge process 1
(pid=732388) process 7 pid 696  len parquets (epochs): 12
(pid=732387) process 9 pid 441  len parquets (epochs): 12
(pid=732386) process 8 pid 698  len parquets (epochs): 12
(pid=732381) exit merge process 2
(pid=732385) exit merge process 3
(pid=732379) process 4 pid 678  len parquets (epochs): 12
(pid=732388) exit merge process 7
(pid=732380) process 6 pid 371  len parquets (epochs): 12
(pid=732384) process 0 pid 456  len parquets (epochs): 12
(pid=732386) process 8 pid 413  len parquets (epochs): 12
(pid=732379) exit merge process 4
(pid=732380) exit merge process 6
(pid=732384) exit merge process 0
(pid=732387) process 9 pid 442  len parquets (epochs): 12
(pid=732386) exit merge process 

In [10]:
ray.shutdown()